# **PUNTO 1 : STIMA PROBABILITA' FALLIMENTO**


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import Generator, MT19937
from numpy.random import seed
from numpy.random import rand
from tqdm import tqdm

In [2]:
# Parametri di base
mu = 0.08       # Tasso di rendimento atteso
r = 0.03        # Tasso di interesse privo di rischio
sigma = 0.25    # Volatilità
T = 5           # Orizzonte temporale (in anni)
S0 = 100        # Prezzo iniziale dell'asset rischioso
F = 100         # Floor
B = 80          # Valore iniziale del bond
C = F - B       # Cushion
n_sim = 1_000_000  # Numero di traiettorie da simulare

#Fissiamo il seed per la riproducibilità dei risultati
rng = Generator(MT19937(1))
seed(112358)

# Liste di dt e moltiplicatori m
dt_values = [0.25, 0.5, 1.0] #3 mesi, 6 mesi, 1 anno
m_values = [2, 5]

results = []

for dt in tqdm(dt_values):
    N = int(T / dt)
    interest = np.exp(r * dt) - 1

    # Simulazione Black-Scholes (log-normale)
    eps = rng.standard_normal((n_sim, N))
    S_paths = np.zeros((n_sim, N))
    S_paths[:, 0] = S0
    for i in range(N-1): #soluzione formula chiusa
        S_paths[:, i+1] = (
            S_paths[:, i] *
            np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * eps[:, i])
        )

    # Calcolo probabilità di fallimento per ogni m
    for m in m_values:
        n_s = m * C / S0 #quote asset rischioso
        n_r = (F - m * C) / B #quote asset risk-free
        E0 = n_s * S_paths[:, 0]
        D0 = n_r * B
        F0 = E0 + D0
        F1 = F0*(1 + interest ) + m * C * ( (S_paths[:, 1] - S_paths[:, 0]) / S_paths[:, 0] - interest )
        fail_prob = np.mean(F1 < B*(1+interest*dt))
        results.append({'dt': dt, 'm': m, 'failure_probability': fail_prob})

# Creazione del DataFrame dei risultati
df = pd.DataFrame(results)
df_pivot = df.pivot(index='dt', columns='m', values='failure_probability')
print(f'\n Probabilità di fallimento con {n_sim} di simulazioni MC \n')
df_pivot

100%|██████████| 3/3 [00:06<00:00,  2.21s/it]



 Probabilità di fallimento con 1000000 di simulazioni MC 



m,2,5
dt,,
0.25,0.000000,0.031125
0.50,0.000022,0.086897
1.00,0.002229,0.166484


# **PUNTO 2: STIMA ERRORE MC**

In [3]:
#ATTENZIONE - SIMULAZIONE COMPUTAZIONALMENTE ONEROSA

# Liste di dt e moltiplicatori m
n_sim = 100_000
n_MC = 10_000

rng = Generator(MT19937(1))
seed(112358)

# Risultati provvisori
all_failures = {(dt, m): [] for dt in dt_values for m in m_values}

for z in tqdm(range(n_MC)):
    for dt in dt_values:
        N = int(T / dt)
        interest = np.exp(r * dt) - 1

        # Simulazione Black-Scholes (log-normale)
        eps = rng.standard_normal((n_sim, N))
        S_paths = np.zeros((n_sim, N))
        S_paths[:, 0] = S0
        for i in range(N-1):
            S_paths[:, i+1] = (
                S_paths[:, i] *
                np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * eps[:, i])
            )

        # Calcolo probabilità di fallimento per ogni m
        for m in m_values:
            n_s = m * C / S0
            n_r = (F - m * C) / B
            E0 = n_s * S_paths[:, 0]
            D0 = n_r * B
            F0 = E0 + D0
            F1 = F0*(1 + interest) + m * C * ( (S_paths[:, 1] - S_paths[:, 0]) / S_paths[:, 0] - interest)
            fail_prob = np.mean(F1 <  B*(1+interest))
            all_failures[(dt, m)].append(fail_prob)

# Creazione del DataFrame
records = []
for (dt, m), failures in all_failures.items():
    mean_failure = np.mean(failures)
    std_failure = np.std(failures)
    records.append({
        'dt': dt,
        'm': m,
        'mean_failure_probability': mean_failure,
        'std_failure_probability': std_failure
    })

df_final = pd.DataFrame(records)
df_final['mc_error'] = df_final['std_failure_probability'] / np.sqrt(n_MC)
df_final['confidence_interval'] = (
    df_final['mean_failure_probability'].round(6).astype(str) +
    ' ± ' +
    df_final['std_failure_probability'].round(6).astype(str)
)

df_pivot = df_final.pivot(index='dt', columns='m', values='confidence_interval')

df_pivot

  2%|▏         | 212/10000 [00:55<42:47,  3.81it/s]


KeyboardInterrupt: 

In [ ]:
df_final

# **PUNTO 3 CONFRONTO VAR CPPI vs RISK ASSET**

In [ ]:
# Parametri di base
T = 1           # Orizzonte temporale (in anni)
S0 = 100        # Prezzo iniziale dell'asset rischioso
F = 100         # Floor
B = 80          # Valore iniziale del bond
C = F - B       # Cushion
n_sim = 10_000_000 # Numero di traiettorie da simulare
dt = 0.25       # Passo temporale
m = 5           # Moltiplicatore

# Simulazione dei percorsi
N = int(T / dt) + 1
t = np.linspace(0, T, N)
Bond_path = B * np.exp(r * t)

rng = Generator(MT19937(1))
seed(112358)

# Simulazione Black-Scholes (log-normale)
eps = rng.standard_normal((n_sim, N))
S_paths = np.zeros((n_sim, N))
S_paths[:, 0] = S0
for i in range(N-1):  # soluzione formula chiusa
    S_paths[:, i+1] = (
        S_paths[:, i] *
        np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * eps[:, i])
    )

losses = []
VaR = []
F1 = F
for j in range(1,N):
  n_s = m * C / S_paths[:, j-1] #aggiorno le quote del portafoglio con l'asset al prezzo precedente
  n_r = (F1 - m * C) / Bond_path[j-1]
  E1 = n_s * S_paths[:, j]
  D1 = n_r * Bond_path[j]
  F1 = E1 + D1
  losses.append(E1 - Bond_path[j]) #Value at risk solo per la parte rischiosa E1
  VaR.append(np.percentile(losses[j-1], 0.5))  # VaR al 99,5%
  print(f'Value-at-Risk (VaR) al 99,5% al tempo {(j) * dt} portfoglio con aggiornamento quote:  {VaR[-1]:.2f}')

In [ ]:
# Calcolo del valore finale del portafoglio con quote costanti iniziali
n_s = m * C / S0
n_r = (F - m * C) / B
E1_fix = n_s * S_paths[:, -1]   # Valore finale dell'asset rischioso
D1_fix = n_r * Bond_path[-1]    # Valore finale del bond
F1_fix = E1_fix + D1_fix        # Valore finale del portafoglio

# Calcolo delle perdite
losses_fix = E1_fix - Bond_path[-1]  # Perdita della componente rischiosa rispetto al valore del bond

# Calcolo del VaR al 99,5%
VaR_fix = np.percentile(losses_fix, 0.5)  # VaR al 99,5%

# Output del risultato
print(f'Value-at-Risk (VaR) al 99,5% con portfoglio con quote costanti: {VaR_fix:.2f}')

# **PUNTO 4 - Modello di Kou**

La simulazione dell’asset rischioso sembra corretta e coerente con il modello a salti di Kou. Tuttavia, il problema principale riguarda la corretta valutazione delle probabilità condizionate di fallimento per ciascuna traiettoria.

Abbiamo tentato due approcci diversi per il problema.

Abbiamo inizialmente provato a gestire ogni singola strategia marcando con 1 i casi in cui il portafoglio fallisce in un determinato istante di tempo. In questo modo, otteniamo un vettore riga di dimensione n_sim per ciascuno degli N passi temporali. Combinando questi vettori si costruisce una matrice di fallimento, dove ogni colonna rappresenta una traiettoria e ogni riga un istante temporale.

Successivamente, abbiamo definito la probabilità di fallimento come la frazione di colonne (cioè traiettorie) la cui somma è diversa da zero, ovvero quelle che presentano almeno un fallimento durante l’intero orizzonte. Questa definizione, però, portava a risultati controintuitivi: ad esempio, una probabilità di fallimento di circa il 23% anche per il caso m = 2, che dovrebbe essere più conservativo.


In [13]:
# --- Parametri di base ---
mu = 0.101        # drift asset rischioso
sigma = 0.161     # volatilità
r = 0.03          # tasso risk-free
T = 5             # orizzonte (anni)
S0 = 1000         # prezzo iniziale
F = 1000          # floor
B = 800           # bond iniziale
C = F - B         # cushion
n_sim = 10000     # numero traiettorie

dt = 1/252        # passo giornaliero
m = 2             # moltiplicatore

# numero di step e punti
N_steps = int(T/dt)
N = N_steps + 1

# RNG
rng = Generator(MT19937(1))
seed(112358)

# griglia e bond path
t = np.linspace(0, T, N)
Bond_path = B * np.exp(r * t)

def generate_kou_jumps(n_sim, n_steps, lambda_param, p, eta_plus, eta_minus, dt):
    counts   = rng.poisson(lam=lambda_param * dt, size=(n_sim, n_steps))
    counts_p = rng.binomial(counts, p)
    counts_m = counts - counts_p
    # jump medio = eta, quindi scale=eta
    jump_p   = rng.gamma(shape=counts_p, scale=eta_plus)
    jump_m   = rng.gamma(shape=counts_m, scale=eta_minus)
    return jump_p - jump_m

# Genera i salti (n_sim x (N-1))
# Parametri di lambda, p, eta+, eta- sono presi dallo Shangai Composite, a pagina 394 di Cont et al
jump_matrix = generate_kou_jumps(
    n_sim=n_sim,
    n_steps=N-1,
    lambda_param=39.1,
    p=0.462,
    eta_plus=0.0167,
    eta_minus=0.0175,
    dt=dt
)

# Simula i percorsi di S (drift + vol + jump)
eps = rng.standard_normal((n_sim, N-1))
S_paths = np.zeros((n_sim, N))
S_paths[:,0] = S0

for i in range(N-1):
    S_paths[:, i+1] = (
        S_paths[:, i] *
        np.exp(
            (mu - 0.5*sigma**2)*dt
            + sigma*np.sqrt(dt)*eps[:, i]
            + jump_matrix[:, i]
        )
    )

# fail_matrix[j-1, i] = 1 se la i-esima traiettoria fallisce al passo j
fail_matrix = []

n_s = m * C / S0
n_r = (F - m * C) / B
F_prev = n_s * S_paths[:, 0] + n_r * Bond_path[0]

for j in range(1, N):
    # ribilancio su ogni traiettoria
    n_s = m * C / S_paths[:, j-1]
    n_r = (F_prev - m * C) / Bond_path[j-1]

    # valore portafoglio a passo j
    F_j = n_s * S_paths[:, j] + n_r * Bond_path[j]
    F_prev = F_j.copy()

    # vettore 0/1 per questo step
    just_failed = (F_j < Bond_path[j])
    row = np.where(just_failed, 1, 0)
    fail_matrix.append(row)

# Trasforma in array e calcola le statistiche
fail_matrix = np.vstack(fail_matrix)     # shape = (N-1, n_sim)
fail_counts = fail_matrix.sum(axis=0)    # quante volte fallisce ciascuna traiettoria

# percentuale di traiettorie che NON falliscono mai
percent_no_failures = np.mean(fail_counts == 0) * 100
percent_failures    = 100 - percent_no_failures

print(f"Percentuale di traiettorie senza alcun fallimento: {percent_no_failures:.2f}%")
print(f"Percentuale di traiettorie con almeno un fallimento: {percent_failures:.2f}%")

Percentuale di traiettorie senza alcun fallimento: 77.43%
Percentuale di traiettorie con almeno un fallimento: 22.57%


In questo secondo caso consideriamo solo le singole probabilità di fallimento per ogni strategia a ogni step.

Il problema, a nostro avviso, risiede nel fatto che una traiettoria può fallire in uno step ma recuperare in quelli successivi. In tal caso, non teniamo conto del fatto che il fallimento, una volta avvenuto, dovrebbe compromettere l’intera strategia. Le probabilità di fallimento successive non risultano quindi condizionate al fallimento già avvenuto, generando una stima errata.

In [14]:
# Parametri di base - Shangai Composite pag.394 Cont et al.

mu = 0.101        # drift asset rischioso
sigma = 0.161     # volatilità
r = 0.03          # tasso risk-free
T = 5             # orizzonte (anni)
S0 = 1000         # prezzo iniziale
F = 1000          # floor
B = 800           # bond iniziale
C = F - B         # cushion
n_sim = 10000    # numero traiettorie

dt = 1/252                  # passo giornaliero
m_values = np.arange(2,7)   # moltiplicatori

N_steps = int(T/dt)
N = N_steps + 1

# Seed
rng = Generator(MT19937(1))
seed(112358)

t = np.linspace(0, T, N)
Bond_path = B * np.exp(r * t)

def generate_kou_jumps(n_sim, n_steps, lambda_param, p, eta_plus, eta_minus, dt):
    # scala lambda al passo dt
    counts    = rng.poisson(lam=lambda_param * dt, size=(n_sim, n_steps))
    counts_p  = rng.binomial(counts, p)
    counts_m  = counts - counts_p

    jump_p    = rng.gamma(shape=counts_p, scale=eta_plus)
    jump_m    = rng.gamma(shape=counts_m, scale=eta_minus)

    return jump_p - jump_m


# Genera i salti solo per N-1 intervalli
jump_matrix = generate_kou_jumps(
    n_sim=n_sim,
    n_steps=N-1,
    lambda_param=39.1,
    p=0.462,
    eta_plus=0.0167,
    eta_minus=0.0175,
    dt=dt
)

# Simula S con drift+vol+jumps
eps = rng.standard_normal((n_sim, N-1))
S_paths = np.zeros((n_sim, N))
S_paths[:,0] = S0
for i in range(N-1):
    S_paths[:, i+1] = (
        S_paths[:, i] *
        np.exp(
            (mu - 0.5*sigma**2)*dt
            + sigma*np.sqrt(dt)*eps[:, i]
            + jump_matrix[:, i]
        )
    )

# Ora ciclo su m e su ogni step j per calcolare failure prob passo-passo
records = []

for m in m_values:
    # inizializzo il valore del portafoglio a t=0
    n_s = m * C / S0
    n_r = (F - m * C) / B
    F_j = n_s * S_paths[:, 0] + n_r * Bond_path[0]

    # per ogni sub-step j=1...N-1
    for j in range(1, N):
        # ribilancio usando F_prev
        n_s = m * C / (S_paths[:, j-1])
        n_r = (F_j - m * C) / Bond_path[j-1]

        # valore portafoglio al passo j
        E_j = n_s * S_paths[:, j]
        D_j = n_r * Bond_path[j]
        F_j = E_j + D_j

        # failure probability al passo j
        fail_prob = np.mean(F_j < Bond_path[j])

        records.append({
            't': t[j],
            'step': j,
            'm': m,
            'failure_probability': fail_prob
        })


df = pd.DataFrame(records)

# 2) Pivot usando "step" come indice
df_pivot = df.pivot(index='step', columns='m', values='failure_probability')

print(f'\nProbabilità di fallimento con {n_sim} simulazioni:\n')
df_pivot


Probabilità di fallimento con 10000 simulazioni:



m,2,3,4,5,6
step,,,,,
1,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...
1256,0.1109,0.1931,0.2464,0.2808,0.3053
1257,0.1108,0.1922,0.2469,0.2822,0.3051
1258,0.1101,0.1924,0.2481,0.2822,0.3063
